# Helper function to convert number to friendly string

In [65]:
def human_readable(num):
    num = float(num)  # make sure it’s numeric
    if num >= 1_000_000_000_000:   # Trillions
        return f"{num/1_000_000_000_000:.1f} T"
    elif num >= 1_000_000_000:     # Billions
        return f"{num/1_000_000_000:.1f} B"
    elif num >= 1_000_000:         # Millions
        return f"{num/1_000_000:.1f} M"
    elif num >= 1_000:             # Thousands
        return f"{num/1_000:.0f} K"
    else:
        return str(num)


# Getting GDP (in US dollars)

In [85]:
import requests
import pandas as pd

BASE = "https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"


COUNTRIES = ["USA", "MEX", "GBR"]
INDICATORS = {
    "NY.GDP.MKTP.CD": "gdp_usd",
    "SP.POP.TOTL": "population",
    "MS.MIL.XPND.GD.ZS": "mil_exp_pct_gdp",
    "MS.MIL.XPND.CD": "mil_exp_usd",
}

START, END = 2012, 2023

def fetch_indicator(country, start, end):
    merged = None
    for code, colname in indicators.items():
        url = BASE.format(country=country, indicator=code)
        params = {"format": "json", "date": f"{start}:{end}", "per_page": 20000}
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        payload = r.json()
        rows = payload[1] if isinstance(payload, list) and len(payload) > 1 else []

        df = pd.DataFrame([
            {
                "country": row["country"]["value"],
                "iso3": row["countryiso3code"],
                "year": int(row["date"]),
                colname: row["value"],
            }
            for row in rows if row.get("value") is not None
        ])

        # merge indicator onto main DataFrame
        if merged is None:
            merged = df
        else:
            merged = pd.merge(merged, df, on=["country", "iso3", "year"], how="outer")

    merged['gdp_string'] = merged['gdp_usd'].apply(human_readable)
    merged['population_string'] = merged['population'].apply(human_readable)
    return merged.sort_values("year").reset_index(drop=True)


# ---------------- RUN ----------------
frames = [fetch_indicator(c, START, END) for c in COUNTRIES]
df = pd.concat(frames).sort_values(["year"]).reset_index(drop=True)

df.head(50)
# df.to_csv("gdp_countries.csv", index=False)


,country,iso3,year,gdp_usd,population,mil_exp_pct_gdp,mil_exp_usd,gdp_string,population_string
0,United States,USA,2012,1.625397e+13,314339099,4.461710,7.252050e+11,16.3 T,314.3 M
1,Mexico,MEX,2012,1.255110e+12,116818208,0.455502,5.717036e+09,1.3 T,116.8 M
2,United Kingdom,GBR,2012,2.707090e+12,63711000,2.417817,6.545249e+10,2.7 T,63.7 M
3,Mexico,MEX,2013,1.327436e+12,118343573,0.487643,6.473144e+09,1.3 T,118.3 M
4,United Kingdom,GBR,2013,2.784854e+12,64139000,2.292320,6.383772e+10,2.8 T,64.1 M
5,United States,USA,2013,1.688068e+13,316726282,4.023706,6.792290e+11,16.9 T,316.7 M
6,United States,USA,2014,1.760814e+13,319257560,3.678918,6.477890e+11,17.6 T,319.3 M
7,Mexico,MEX,2014,1.364508e+12,119784261,0.495321,6.758694e+09,1.4 T,119.8 M
8,United Kingdom,GBR,2014,3.064708e+12,64620000,2.184485,6.699547e+10,3.1 T,64.6 M
9,United Kingdom,GBR,2015,2.927911e+12,65088000,2.048908,5.999021e+10,2.9 T,65.1 M
